In [604]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from arch.univariate import ConstantMean, GARCH, StudentsT, Normal

import warnings

warnings.filterwarnings("ignore")  # nuke all warnings



In [605]:
X_train = pd.read_csv('X_train_enriched.csv')
X_val = pd.read_csv('X_val_enriched.csv')
y_train = pd.read_csv('y_train.csv')['market_forward_excess_returns']
y_val = pd.read_csv('y_val.csv')['market_forward_excess_returns']

In [606]:
X_features_train = X_train[-252:]
X_train = X_train[:-252]
y_features_train = y_train[-252:]
y_train = y_train[:-252]

In [607]:
X_train_cleaned = X_train.ffill().bfill()
X_features_train_cleaned = X_features_train.ffill().bfill()
X_val_cleaned = X_val.ffill().bfill()


In [608]:
cols_with_na_train = X_train_cleaned.columns[X_train_cleaned.isna().any()]
cols_with_na_val = X_val_cleaned.columns[X_val_cleaned.isna().any()]
cols_with_na_features_train = X_features_train_cleaned.columns[X_features_train_cleaned.isna().any()]
all_cols_with_na = set(cols_with_na_train).union(set(cols_with_na_val)).union(set(cols_with_na_features_train))
drop_cols = list(all_cols_with_na)
X_train_cleaned = X_train_cleaned.drop(columns=drop_cols)
X_val_cleaned = X_val_cleaned.drop(columns=drop_cols)
X_features_train_cleaned = X_features_train_cleaned.drop(columns=drop_cols)

In [609]:
X_train_cleaned

,D1_rollmean5,D2_rollmean5,D3_rollmean5,D4_rollmean5,D5_rollmean5,D6_rollmean5,D7_rollmean5,D8_rollmean5,D9_rollmean5,E1_rollmean5,...,V11_rollstd20,V12_rollstd20,V13_rollstd20,V2_rollstd20,V3_rollstd20,V4_rollstd20,V5_rollstd20,V6_rollstd20,V7_rollstd20,V8_rollstd20
0,0.0,0.0,0.0,1.0,0.4,0.0,0.0,0.0,0.6,2.326470,...,0.000000,0.000000,0.205994,0.006483,0.064361,0.006245,0.235007,0.000000,0.307308,0.000000
1,0.0,0.0,0.0,1.0,0.4,0.0,0.0,0.0,0.6,2.326470,...,0.000000,0.000000,0.205994,0.006483,0.064361,0.006245,0.235007,0.000000,0.307308,0.000000
2,0.0,0.0,0.0,1.0,0.4,0.0,0.0,0.0,0.6,2.326470,...,0.000000,0.000000,0.205994,0.006483,0.064361,0.006245,0.235007,0.000000,0.307308,0.000000
3,0.0,0.0,0.0,1.0,0.4,0.0,0.0,0.0,0.6,2.326470,...,0.000000,0.000000,0.205994,0.006483,0.064361,0.006245,0.235007,0.000000,0.307308,0.000000
4,0.0,0.0,0.0,1.0,0.4,0.0,0.0,0.0,0.6,2.326470,...,0.000000,0.000000,0.205994,0.006483,0.064361,0.006245,0.235007,0.000000,0.307308,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6935,0.0,0.0,0.0,0.0,0.4,0.0,0.2,0.0,0.6,0.720845,...,0.104215,0.214891,0.068393,0.057615,0.141045,0.054971,0.523733,0.211907,0.107330,0.051271
6936,0.0,0.0,0.0,0.0,0.2,0.0,0.2,0.0,0.4,0.719789,...,0.109211,0.223626,0.066307,0.063395,0.137276,0.047187,0.526851,0.200598,0.105251,0.052269
6937,0.0,0.0,0.2,0.0,0.0,0.0,0.2,0.0,0.2,0.718734,...,0.110043,0.222056,0.066681,0.062737,0.142143,0.042914,0.499615,0.186592,0.105139,0.049157
6938,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.717679,...,0.111068,0.215146,0.069717,0.062666,0.142084,0.031799,0.485718,0.174361,0.109659,0.048913


In [610]:
def feature_selection(X_tr, y_tr, X_va, y_va, *, k=30, top_corr=60,
                     use_extratrees=True, val_last=252, n_repeats=5, seed=42, w_corr=0.6, w_perm=0.4):
    # 1) keep a small validation slice
    if val_last is not None and len(X_va) > val_last:
        Xv = X_va.iloc[-val_last:].astype(np.float32).copy()
        yv = y_va.iloc[-val_last:].to_numpy()
    else:
        Xv = X_va.astype(np.float32).copy()
        yv = y_va.to_numpy()

    # 2) univariate Pearson on TRAIN; take top N
    corr_abs = X_tr.apply(lambda c: np.corrcoef(c, y_tr)[0,1], axis=0).abs().fillna(0.0)
    cand = corr_abs.sort_values(ascending=False).head(min(top_corr, X_tr.shape[1])).index.tolist()

    # 3) small, fast tree on TRAIN
    Tree = ExtraTreesRegressor if use_extratrees else RandomForestRegressor
    tree = Tree(
        n_estimators=100, max_depth=8, min_samples_leaf=0.01, max_features=0.7,
        n_jobs=-1, random_state=seed
    ).fit(X_tr[cand].astype(np.float32), y_tr.to_numpy())

    # 4) permutation importance on VALID (few repeats, parallel)
    pi = permutation_importance(tree, Xv[cand], yv, n_repeats=n_repeats,
                                random_state=seed, scoring="neg_mean_squared_error", n_jobs=-1)
    perm = pd.Series(np.clip(pi.importances_mean, 0, None), index=cand)

    # 5) combine by rank (robust to scaling)
    r_corr = corr_abs.loc[cand].rank(ascending=False)
    r_perm = perm.rank(ascending=False)
    score = (w_corr * r_corr + w_perm * r_perm).sort_values(ascending=False)

    selected = score.index[:min(k, len(score))].tolist()
    summary = pd.DataFrame({"corr_abs": corr_abs.loc[cand], "perm": perm, "score_rank": score}).loc[score.index]
    return selected, summary

In [611]:
def hit_rate(y_true, y_pred, *, dropna: bool = True, margin: float = 0.0, count_ties: bool = False) -> float:
    """
    Sign accuracy (hit rate): fraction of times sign(y_pred) == sign(y_true).

    Parameters
    ----------
    y_true, y_pred : array-like
        Equal-length sequences of numbers.
    dropna : bool, default True
        If True, drop any pair with NaN in either array.
        If False and NaNs are present, returns np.nan.
    margin : float, default 0.0
        Treat predictions with |y_pred| <= margin as 0 (neutral band).
    count_ties : bool, default False
        If False, exclude any pair where sign is 0 on either side.
        If True, include pairs with sign==0 and count them as correct
        only when both are 0.

    Returns
    -------
    hit : float
        Proportion in [0,1], or np.nan if no eligible pairs.
    """
    a = np.asarray(y_true, dtype=float).flatten()
    b = np.asarray(y_pred, dtype=float).flatten()

    if a.shape != b.shape:
        raise ValueError("y_true and y_pred must have the same shape")

    mask = np.isfinite(a) & np.isfinite(b)
    if not dropna and not mask.all():
        return np.nan
    a = a[mask]
    b = b[mask]

    # Apply neutral band to predictions
    if margin > 0:
        b = b.copy()
        b[np.abs(b) <= margin] = 0.0

    s_true = np.sign(a)
    s_pred = np.sign(b)

    if count_ties:
        eligible = np.ones_like(s_true, dtype=bool)
    else:
        eligible = (s_true != 0) & (s_pred != 0)

    if not np.any(eligible):
        return np.nan

    hits = (s_true[eligible] == s_pred[eligible]).mean()
    return float(hits)

In [612]:
def pearson_corr(y_true, y_pred):
    """
    Pearson correlation coefficient between y_true and y_pred.

    Parameters
    ----------
    y_true, y_pred : array-like
        Equal-length sequences of numbers.

    Returns
    -------
    corr : float
        Pearson correlation coefficient in [-1,1], or np.nan if undefined.
    """
    a = np.asarray(y_true, dtype=float).flatten()
    b = np.asarray(y_pred, dtype=float).flatten()

    if a.shape != b.shape:
        raise ValueError("y_true and y_pred must have the same shape")

    mask = np.isfinite(a) & np.isfinite(b)
    a = a[mask]
    b = b[mask]

    if len(a) == 0:
        return np.nan

    corr, _ = pearsonr(a, b)
    return float(corr)

In [613]:
def ridge_train(X_train, y_train, features = None, alpha=1.0):
    """
    Train a Ridge regression model and evaluate on validation set.

    Parameters
    ----------
    X_train : pd.DataFrame
        Training features.
    y_train : pd.Series
        Training target.
   
    alpha : float, default 1.0
        Regularization strength for Ridge regression.

    Returns
    -------
    model : Ridge
        Trained Ridge regression model.
    
    """
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('ridge', Ridge(alpha=alpha))
    ])

    print(features)

    if features:
        model.fit(X_train[features], y_train)
    else:
        model.fit(X_train, y_train)

    return model

In [614]:
def logistic_train(X_train, y_train, features = None, C=1.0):
    """
    Train a Logistic regression model and evaluate on validation set.

    Parameters
    ----------
    X_train : pd.DataFrame
        Training features.
    y_train : pd.Series
        Training target.
   
    C : float, default 1.0
        Inverse of regularization strength for Logistic regression.

    Returns
    -------
    model : LogisticRegression
        Trained Logistic regression model.
    
    """
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('logistic', LogisticRegression(C=C, max_iter=1000))
    ])

    if features:
        model.fit(X_train[features], y_train)
    else:
        model.fit(X_train, y_train)

    return model

In [615]:
def trees_train(X_train, y_train, features = None, type='RandomForest'):
    """
    Train a Tree Regressor and evaluate on validation set.

    Parameters
    ----------
    X_train : pd.DataFrame
        Training features.
    y_train : pd.Series
        Training target.
   
    n_estimators : int, default 100
        Number of trees in the forest.
    
    max_depth : int or None, default None
        Maximum depth of the tree.

    random_state : int, default 42
        Random seed for reproducibility.

    Returns
    -------
    model : Regressor
    
    """
    
    if type == 'RandomForest':
        model = RandomForestRegressor(n_estimators=100,
            max_depth=8,
            min_samples_leaf=0.01,     # 1% of samples per leaf (robust)
            min_samples_split=0.02,
            max_features=0.7,
            bootstrap=True,
            n_jobs=-1, random_state=42)
        
    elif type == 'ExtraTrees':
        model = ExtraTreesRegressor(
            n_estimators=100,
            max_depth=8,
            min_samples_leaf=0.01,
            min_samples_split=0.02,
            max_features=0.7,
            bootstrap=False,
            n_jobs=-1, random_state=42
        )

    elif type == 'XGBoost':
        model = XGBRegressor(
            n_estimators=100,
            learning_rate=0.10,
            max_depth=4,
            subsample=0.7,
            colsample_bytree=0.7,
            min_child_weight=10,       # combats noise
            reg_lambda=2.0,
            objective="reg:squarederror",
            n_jobs=-1, random_state=42
        )
    elif type == 'LightGBM':
        model = LGBMRegressor(
            verbosity = -1,
            n_estimators=100,
            learning_rate=0.10,
            max_depth=6,
            num_leaves=31,             # <= 2^max_depth for safety
            min_data_in_leaf=100,      # robust on small-signal data
            feature_fraction=0.7,
            bagging_fraction=0.7,
            bagging_freq=1,
            lambda_l2=5.0,
            extra_trees=True,          # adds randomness like ExtraTrees
            n_jobs=-1, random_state=42
        )
    if features:
        model.fit(X_train[features], y_train)
    else:
        model.fit(X_train, y_train)

    return model

In [616]:
def linear_calibrate(pred_train, y_train, *, dropna: bool = True, fit_intercept: bool = True):
    """
    Fit a scikit-learn LinearRegression on TRAIN predictions:
        y = alpha + beta * pred

    Returns a Pipeline that reshapes 1D inputs and applies the fitted LinearRegression.
    You can call .predict(...) on it with a 1D array/list/Series of predictions.

    Parameters
    ----------
    pred_train : array-like, shape (n_samples,)
        Model predictions on the training window (e.g., your tree's outputs).
    y_train : array-like, shape (n_samples,)
        Realized targets on the training window (e.g., next-day returns).
    dropna : bool, default True
        Drop pairs with NaN/Inf before fitting. If False and NaNs exist, raises ValueError.
    fit_intercept : bool, default True
        Passed to LinearRegression.

    Returns
    -------
    model : sklearn Pipeline
        Use model.predict(new_pred) to get calibrated predictions.
        Access alpha/beta via:
            alpha = model.named_steps['lr'].intercept_
            beta  = model.named_steps['lr'].coef_[0]
    """
    x = np.asarray(pred_train, dtype=float).flatten()
    y = np.asarray(y_train, dtype=float).flatten()
    if x.shape != y.shape:
        raise ValueError("pred_train and y_train must have the same shape")

    mask = np.isfinite(x) & np.isfinite(y)
    if not dropna and not mask.all():
        raise ValueError("NaNs/Infs present; set dropna=True to filter them out.")
    x, y = x[mask], y[mask]
    if x.size < 2:
        raise ValueError("Not enough data to calibrate (need ≥2 finite pairs).")

    # Pipeline so you can pass 1D arrays to .predict() without manual reshape
    model = Pipeline(steps=[
        ("reshape", FunctionTransformer(lambda z: np.asarray(z, dtype=float).reshape(-1, 1), validate=False)),
        ("lr", LinearRegression(fit_intercept=fit_intercept)),
    ])
    model.fit(x, y)
    return model


In [617]:
def predict(model, X, calibrate_model):
    """
    Generate predictions using the trained model.

    Parameters
    ----------
    model : Trained model
        The trained regression model.
    X : pd.DataFrame
        Features for prediction.

    Returns
    -------
    y_pred : np.ndarray
        Predicted values.
    """
    y_pred = model.predict(X)

    calibrated_pred = calibrate_model.predict(y_pred)
    return calibrated_pred

In [618]:
regression_features, regression_feature_summary = feature_selection(X_train_cleaned, y_train, X_features_train_cleaned, y_features_train, w_corr=0.9, w_perm=0.1, k = 30, top_corr=30)

In [619]:
ridge = ridge_train(X_train_cleaned, y_train, features = regression_features, alpha=1.0)
ridge_predict_train = ridge.predict(X_train_cleaned[regression_features])
calibrate_model = linear_calibrate(ridge_predict_train, y_train)
ridge_predict_val = predict(ridge, X_val_cleaned[regression_features], calibrate_model)
ridge_hit_rate = hit_rate(y_val, ridge_predict_val)
ridge_pearson = pearson_corr(y_val, ridge_predict_val)
print("Ridge Hit Rate:", ridge_hit_rate, "Pearson Correlation:", ridge_pearson)

['D4_rollmean20', 'V13_rollmean20', 'D6_rollstd5', 'V5_rollstd20', 'D9_rollstd5', 'I8_rollstd20', 'D4_rollmean5', 'P10_rollstd5', 'I2_rollmean20', 'E8_rollstd5', 'P8_rollstd5', 'M17_rollmean5', 'M17_rollmean20', 'S2_rollmean20', 'I8_rollstd5', 'P7_rollmean5', 'I2_rollmean5', 'D9_rollmean5', 'D6_rollmean5', 'D7_rollstd5', 'D7_rollmean5', 'E12_rollmean5', 'V13_rollmean5', 'E12_rollmean20', 'S2_rollmean5', 'E11_rollmean5', 'E11_rollmean20', 'M4_rollmean5', 'S5_rollmean20', 'S5_rollmean5']
Ridge Hit Rate: 0.5139043381535039 Pearson Correlation: 0.026745103772223253


In [620]:
logistic_y = [1 if x > 0 else 0 for x in list(np.asarray(y_train).flatten())]

In [621]:
logistic = logistic_train(X_train_cleaned, pd.Series(logistic_y), features = regression_features, C=1.0)
logistic_predict_train = logistic.predict_proba(X_train_cleaned[regression_features])
logistic_calibrate_model = linear_calibrate(logistic_predict_train[:,1], y_train)
logistic_predict_val = predict(logistic, X_val_cleaned[regression_features], logistic_calibrate_model)
logistic_hit_rate = hit_rate(y_val, logistic_predict_val)
logistic_pearson = pearson_corr(y_val, logistic_predict_val)
print("Logistic Hit Rate:", logistic_hit_rate, "Pearson Correlation:", logistic_pearson)

Logistic Hit Rate: 0.5350389321468298 Pearson Correlation: 0.043807784585532215


In [622]:
btree_features, btree_feature_summary = feature_selection(X_train_cleaned, y_train, X_features_train_cleaned, y_features_train, w_corr=0.7, w_perm=0.3, k = 200, top_corr=200)

In [623]:
lightgbm = trees_train(X_train_cleaned, y_train, features=btree_features, type='LightGBM')
lightgbm_predict_train = lightgbm.predict(X_train_cleaned[btree_features])
lightgbm_calibrate_model = linear_calibrate(lightgbm_predict_train, y_train)
lightgbm_predict_val = predict(lightgbm, X_val_cleaned[btree_features], lightgbm_calibrate_model)
lightgbm_hit_rate = hit_rate(y_val, lightgbm_predict_val)
lightgbm_pearson = pearson_corr(y_val, lightgbm_predict_val)
print("LightGBM Hit Rate:", lightgbm_hit_rate, "Pearson Correlation:", lightgbm_pearson)   

LightGBM Hit Rate: 0.5255839822024472 Pearson Correlation: 0.047444636426307404


In [624]:
tree_features, tree_feature_summary = feature_selection(X_train_cleaned, y_train, X_features_train_cleaned, y_features_train, w_corr=0.7, w_perm=0.3, k = 30, top_corr=100)

In [625]:
rf = trees_train(X_train_cleaned, y_train, features=tree_features, type='RandomForest')
rf_predict_train = rf.predict(X_train_cleaned[tree_features])
rf_calibrate_model = linear_calibrate(rf_predict_train, y_train)        
rf_predict_val = predict(rf, X_val_cleaned[tree_features], rf_calibrate_model)
rf_hit_rate = hit_rate(y_val, rf_predict_val)
rf_pearson = pearson_corr(y_val, rf_predict_val)
print("Random Forest Hit Rate:", rf_hit_rate, "Pearson Correlation:", rf_pearson)

Random Forest Hit Rate: 0.4949944382647386 Pearson Correlation: 0.05187643236790405


In [626]:
ensemble_val = (0.3 * logistic_predict_val + 0.4 * lightgbm_predict_val + 0.3 * rf_predict_val)
essemble_hit_rate = hit_rate(y_val, ensemble_val)
ensemble_pearson = pearson_corr(y_val, ensemble_val)
print("Ensemble Hit Rate:", essemble_hit_rate, "Pearson Correlation:", ensemble_pearson)

Ensemble Hit Rate: 0.5367074527252503 Pearson Correlation: 0.058462083308470844


In [627]:
def ewma_var(lam, r, v0):
    v = np.empty_like(r)
    v[0] = v0
    for t in range(1, r.size):
        v[t] = lam*v[t-1] + (1-lam)*r[t-1]**2
    return v

In [628]:
def train_ewma_variance(r_train,
                        lam_grid = None,
                        criterion = "qlike",
                        df_t: int = 8):
    """
    Choose EWMA lambda on TRAIN (no leakage).
    Returns a dict {"lam": ..., "init_var": ...}.
    """
    r = np.asarray(r_train, float).reshape(-1)
    if lam_grid is None:
        lam_grid = np.linspace(0.90, 0.995, num = 10)

    v0 = np.var(r[:max(50, len(r)//5)]) if r.size else 0.0

    best = (np.inf, lam_grid[0])
    m0 = max(20, int(0.1*len(r)))  # drop warmup in scoring
    i = 0
    for lam in lam_grid:
        v = ewma_var(lam, r, v0)
        rr, vv = r[m0:], v[m0:]
        if criterion in ("qlike", "nll_norm"):
            loss = np.mean(np.log(vv) + (rr**2)/vv)
        elif criterion == "nll_t":
            nu = float(df_t)
            loss = np.mean(np.log(vv) + (nu+1)*np.log1p((rr**2)/(nu*vv)))
        elif criterion == "mse":
            loss = np.mean((rr**2 - vv)**2)
        else:
            raise ValueError("criterion must be one of {'qlike','nll_norm','nll_t','mse'}")
        if loss < best[0]:
            best = (loss, lam)
        i += 1

    return {"lam": float(best[1]), "init_var": float(v0)}

In [629]:
def train_garch11(r_train,
                  dist = "t"):
    """
    Fit ConstantMean + GARCH(1,1) on TRAIN. Returns a dict with fitted params & dist.
    (We store the fitted result to reuse its parameters in validation forecasts.)
    """
    r = np.asarray(r_train, float).reshape(-1)
    am = ConstantMean(r * 100.0)              # scale to percent for stability
    am.volatility = GARCH(1, 0, 1)
    am.distribution = StudentsT() if dist == "t" else Normal()
    res = am.fit(disp="off")

    model = {
        "dist": dist,
        "params": res.params,                 # pandas Series of parameters
        "scale": 100.0
    }
    return model


In [630]:
def train_tree_variance_qlike_surrogate(X_train,
                                        r_train,
                                        *,
                                        n_estimators: int = 300,
                                        max_depth: int = 8,
                                        random_state: int = 42):
    """
    QLIKE surrogate: predict s = log(r^2 + eps) on TRAIN, then variance = exp(ŝ).
    (Minimizing E[(s - log r^2)^2] targets the QLIKE optimum since QLIKE(s)=s + r^2 e^{-s}.)
    Returns fitted ExtraTrees model plus epsilon used.
    """
    eps = 1e-10
    y = np.log(np.asarray(r_train, float).reshape(-1)**2 + eps)
    X = np.asarray(X_train, float)
    model = ExtraTreesRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_leaf=max(1, int(0.01*len(y))),
        max_features=0.7,
        n_jobs=-1,
        random_state=random_state
    ).fit(X, y)
    return {"model": model, "eps": eps}


In [631]:
def apply_ewma_on_valid(train_model: dict,
                        r_train,
                        r_valid):
    """
    Walk-forward EWMA variance for VALID using trained {"lam","init_var"}.
    Returns pd.Series aligned to r_valid.index if available.
    """
    lam = float(train_model["lam"]); v0 = float(train_model["init_var"])
    r_tr = np.asarray(r_train, float).reshape(-1)
    r_va = np.asarray(r_valid, float).reshape(-1)
    idx_va = getattr(r_valid, "index", pd.RangeIndex(len(r_va)))

    r_all = np.concatenate([r_tr, r_va])
    v = np.empty_like(r_all)
    v[0] = v0
    for t in range(1, len(r_all)):
        v[t] = lam*v[t-1] + (1-lam)*r_all[t-1]**2
    return pd.Series(v[len(r_tr):], index=idx_va)

In [632]:
def apply_garch_on_valid(train_model: dict,
                         r_train,
                         r_valid,
                         refit_every = 21):
    """
    Walk-forward GARCH(1,1) variance on VALID.
    - Start from TRAIN; if refit_every is None, use fixed parameters (one-shot fit).
    - If refit_every is int, refit every N days using expanding history (adaptive, no leakage).
    Returns pd.Series aligned to r_valid.index if available.
    """
    dist = train_model.get("dist", "t")
    scale = float(train_model.get("scale", 100.0))
    r_tr = np.asarray(r_train, float).reshape(-1)
    r_va = np.asarray(r_valid, float).reshape(-1)
    idx_va = getattr(r_valid, "index", pd.RangeIndex(len(r_va)))

    var_va = np.empty_like(r_va)
    i = 0
    hist = r_tr.copy()

    # If no refits: fit once and forecast the entire validation
    if refit_every is None:
        am = ConstantMean(hist * scale)
        am.volatility = GARCH(1, 0, 1)
        am.distribution = StudentsT() if dist == "t" else Normal()
        res = am.fit(disp="off")
        fc = res.forecast(horizon=len(r_va), reindex=False)
        var_va[:] = fc.variance.values[-1, :len(r_va)] / (scale**2)
        return pd.Series(var_va, index=idx_va)

    # Periodic refits
    step = max(1, int(refit_every))
    while i < len(r_va):
        h = min(step, len(r_va) - i)
        am = ConstantMean(hist * scale)
        am.volatility = GARCH(1, 0, 1)
        am.distribution = StudentsT() if dist == "t" else Normal()
        res = am.fit(disp="off")
        fc = res.forecast(horizon=h, reindex=False)
        var_va[i:i+h] = fc.variance.values[-1, :h] / (scale**2)
        # append realized returns for next refit (expanding window)
        hist = np.r_[hist, r_va[i:i+h]]
        i += h

    return pd.Series(var_va, index=idx_va)

In [633]:
def apply_tree_on_valid(train_model: dict,
                        X_valid):
    """
    Predict VALID variance from ExtraTrees QLIKE surrogate.
    Returns pd.Series (variance), preserves X_valid index if DataFrame.
    """
    model = train_model["model"]; eps = float(train_model["eps"])
    Xv = np.asarray(X_valid, float)
    s_hat = model.predict(Xv)                # ŝ ≈ E[log r^2 | X]
    var_hat = np.exp(s_hat)                  # v̂ = exp(ŝ)  (>=0)
    idx = getattr(X_valid, "index", pd.RangeIndex(len(var_hat)))
    return pd.Series(var_hat, index=idx)

In [634]:
def qlike(r, vhat, eps=1e-12):
    r = np.asarray(r, float).reshape(-1)
    v = np.asarray(vhat, float).reshape(-1)
    v = np.clip(v, eps, None)
    return float(np.mean(np.log(v) + (r**2)/v))

In [653]:
rf_predict_train = predict(rf, X_train_cleaned[tree_features], rf_calibrate_model)

In [654]:
lightgbm_predict_train = predict(lightgbm, X_train_cleaned[btree_features], lightgbm_calibrate_model)

In [655]:
logistic_predict_train = predict(logistic, X_train_cleaned[features_selected], logistic_calibrate_model)

In [656]:
ensemble_train = (0.3 * rf_predict_train + 0.4 * lightgbm_predict_train + 0.3 * logistic_predict_train)

In [657]:
res_train = y_train - ensemble_train

In [635]:
ewma_model = train_ewma_variance(X_train_cleaned)

In [636]:
ewma_var = apply_ewma_on_valid(ewma_model, y_train, ensemble_val)

In [637]:
qlike(y_val, ewma_var)

-6.38421732464044

In [638]:
garch_model = train_garch11(X_train_cleaned)

In [639]:
garch_var = apply_garch_on_valid(garch_model, y_train, ensemble_val)

In [640]:
qlike(y_val, garch_var)

-7.127822114224793

In [641]:
tree_model = train_tree_variance_qlike_surrogate(X_train_cleaned, y_train)

In [642]:
tree_var = apply_tree_on_valid(tree_model, X_val)

In [643]:
qlike(y_val, tree_var)

-6.646020158798007